In [39]:
import requests

Load environment variables

In [40]:
from dotenv import load_dotenv
import os

In [41]:
# load environment variables
load_dotenv(override=True)

# access environment variables
api_key = os.getenv("API_KEY")

api_key

'CG-eiNjAYrU8wLttTPAzZfZZCJ9'

Create a function for sending requests to the CoinGecko API

In [42]:
# send requests to CoinGecko

def coin_gecko_request(url, params=None):
    headers = {
        "accept": "application/json",
        "x-cg-pro-api-key": api_key
    }
    if params is None:
        response = requests.get(url=url, headers=headers)
    else:
        response = requests.get(url=url, headers=headers, params=params)
    if response.status_code == 200:
        return response
    else:
        raise Exception(f"Error: {response.status_code}, {response.text}")

Get list of coin ids, ordered by market cap

In [85]:
# API endpoint for CoinGecko market data on coins
url = "https://pro-api.coingecko.com/api/v3/coins/markets"

params = {
    "vs_currency": "usd",
    "order": "market_cap_desc",
    "per_page": 100,
    "page": 1
}

response = coin_gecko_request(url=url, params=params)
coin_list = response.json()

# get list of coin ids
coin_ids = [coin['id'] for coin in coin_list]
len(coin_ids)

100

From our list of coins, remove stablecoins, or wrapped-tokens (duplicates of other coins on another ecosystems)

In [86]:
# get the list of categories a coin is labelled with

def get_categories(coin_id):
    # API endpoint for getting all data associated with a coin, including categories
    url = f"https://pro-api.coingecko.com/api/v3/coins/{coin_id}"
    response = coin_gecko_request(url=url)
    return response.json()["categories"]


In [87]:
# remove stablecoins from coin list
for coin_id in coin_ids:
    categories = get_categories(coin_id)
    if ("Stablecoins" in categories) or ("Wrapped-Tokens" in categories):
        coin_ids.remove(coin_id)

if len(coin_ids) > 50:
    coin_ids = coin_ids[:50]

len(coin_ids)

50

Collect price data on our list of coins (over the past 5 years)

In [48]:
from datetime import datetime

In [79]:
# january 1, 2020 in unix timestamp
start_date = "January 1, 2020"
start_date_obj = datetime.strptime(start_date, "%B %d, %Y")
start_timestamp = int(start_date_obj.timestamp())

# january 1, 2025 in unix timestamp
end_date = "January 1, 2025"
end_date_obj = datetime.strptime(end_date, "%B %d, %Y")
end_timestamp = int(end_date_obj.timestamp())

In [89]:
# Get daily close? data for coin list, for the past 5 years (365 * 5 = 1825 days)

def fetch_daily_prices(coin_id, start_timestamp, end_timestamp):
    url = f"https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range" 
    params = {
        'vs_currency': 'usd',
        'from': start_timestamp,
        'to': end_timestamp,      
        'interval': 'daily'    
    }
    response = coin_gecko_request(url=url, params=params)
    daily_prices = response.json()["prices"]
    return daily_prices
    
btc_close = fetch_daily_prices("bitcoin", start_timestamp=start_timestamp, end_timestamp=end_timestamp)
btc_close

[[1577923200000, 7193.7546679601],
 [1578009600000, 6963.853353983485],
 [1578096000000, 7300.34693530511],
 [1578182400000, 7351.377053166627],
 [1578268800000, 7349.972248050628],
 [1578355200000, 7734.675515785289],
 [1578441600000, 8147.415237636371],
 [1578528000000, 8045.789124592971],
 [1578614400000, 7832.795643840387],
 [1578700800000, 8180.513169803805],
 [1578787200000, 8012.085781814529],
 [1578873600000, 8171.401185320944],
 [1578960000000, 8110.647964503772],
 [1579046400000, 8801.871213137745],
 [1579132800000, 8820.85010599613],
 [1579219200000, 8720.91950137424],
 [1579305600000, 8902.568995225329],
 [1579392000000, 8906.902675151074],
 [1579478400000, 8690.380569544623],
 [1579564800000, 8631.253954333763],
 [1579651200000, 8724.818622931474],
 [1579737600000, 8665.436914096897],
 [1579824000000, 8415.956027335538],
 [1579910400000, 8432.096675789418],
 [1579996800000, 8348.665665079325],
 [1580083200000, 8595.728298493115],
 [1580169600000, 8879.335548714553],
 [1580

In [90]:
import pandas as pd

In [91]:
# create dataframe with daily price data for all coins in coin list
combined_df = pd.DataFrame()

for i, coin_id in enumerate(coin_ids):
    close_data = fetch_daily_prices(coin_id, start_timestamp=start_timestamp, end_timestamp=end_timestamp)
    if close_data:
        df = pd.DataFrame(close_data, columns=["timestamp", coin_id]) # create df with columns names as timestamp and coin id
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms") # convert the UNIX timestamp to a real date
        if combined_df.empty:
            combined_df = df
        else:
            combined_df = pd.merge(combined_df, df, on='timestamp', how='outer')

combined_df.head()

,timestamp,bitcoin,ethereum,ripple,solana,binancecoin,dogecoin,cardano,staked-ether,tron,...,render-token,bittensor,kaspa,filecoin,fetch-ai,arbitrum,virtual-protocol,gatechain-token,cosmos,ethena
0,2020-01-02,7193.754668,130.484685,0.192556,NaN,13.685902,0.002026,0.033362,NaN,0.013199,...,NaN,NaN,NaN,NaN,0.038686,NaN,NaN,0.353788,4.381528,NaN
1,2020-01-03,6963.853354,127.045258,0.187433,NaN,13.007171,0.002008,0.032618,NaN,0.012845,...,NaN,NaN,NaN,NaN,0.036687,NaN,NaN,0.343728,4.098819,NaN
2,2020-01-04,7300.346935,133.702649,0.192299,NaN,13.603276,0.002049,0.033967,NaN,0.013371,...,NaN,NaN,NaN,NaN,0.038930,NaN,NaN,0.353166,4.227682,NaN
3,2020-01-05,7351.377053,134.136883,0.192812,NaN,13.794575,0.002189,0.034298,NaN,0.013438,...,NaN,NaN,NaN,NaN,0.040255,NaN,NaN,0.354280,4.253197,NaN
4,2020-01-06,7349.972248,135.005714,0.193816,NaN,14.006361,0.002070,0.034427,NaN,0.013512,...,NaN,NaN,NaN,NaN,0.038340,NaN,NaN,0.386061,4.199192,NaN


In [95]:
# convert to csv
combined_df.to_csv("price-data.csv", index=False)